In [1]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import datetime

In [3]:
#Загружаем данные
train = pd.read_csv("/content/drive/MyDrive/train.csv")
test = pd.read_csv("/content/drive/MyDrive/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")
submission["demand"] = submission["demand"].astype(float)

Формируем обучающую и тестовыю выборки

In [4]:
#Обучающаяя
df = train.copy()
df=df.dropna(subset=['demand'])#Удаляем с пустым demand
df=df.dropna(subset=['PRICE_REGULAR'])#Удаляем с пустой ценой
df["product_id"] = df["product_id"].astype(int)#все в int!
df["location_id"] = df["location_id"].astype(int)
df["period_dt"] = pd.to_datetime(df["period_dt"], dayfirst=True)#На всякий случай в datetime

In [5]:
df=df[(pd.DatetimeIndex(df['period_dt']).month ==12)]#Оставляем только декабрь

In [6]:
#Тестовая
test = pd.read_csv("/content/drive/MyDrive/test.csv")
df1 = test.copy()
df1["period_dt"] = pd.to_datetime(df1["period_dt"], dayfirst=True)

Формируем новые признаки

In [7]:
df['year'] = df['period_dt'].dt.year#Год
df['days'] = df['period_dt'].dt.day#день
df['week_number'] = df['period_dt'].dt.isocalendar().week#номер недели

<ipython-input-7-c2f6d401ab44>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['period_dt'].dt.year#Год
<ipython-input-7-c2f6d401ab44>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days'] = df['period_dt'].dt.day#день
<ipython-input-7-c2f6d401ab44>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Удаляем из тренировочной признаки, которых нет в тестовой

In [8]:
#Целевой признак для обучения
y = df["demand"]
#Формируем обучающие переменные
x = df.drop(['demand', 'PROMO1_FLAG','PROMO2_FLAG','PRICE_REGULAR','PRICE_AFTER_DISC','NUM_CONSULTANT','AUTORIZATION_FLAG','period_dt'], axis=1)

In [9]:
#Разделяем выборки на обучающую и тестовую
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.01)

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
#Конвертируем выборки в тензоры
x_train = tf.convert_to_tensor(x_train.astype(float))
y_train = tf.convert_to_tensor(y_train.astype(float))
x_test = tf.convert_to_tensor(x_test.astype(float))
y_test = tf.convert_to_tensor(y_test.astype(float))

In [12]:
#Вычисляем нормы по переменным
x_norm = tf.linalg.norm(x_train, ord=np.inf, axis=0)
y_norm = tf.linalg.norm(y_train, ord=np.inf, axis=0)

In [13]:
#Нормируем выборки
x_train = x_train / x_norm
y_train = y_train / y_norm

x_test = x_test / x_norm
y_test = y_test / y_norm

In [14]:
#Формируем модель нейросети
model = keras.Sequential()
model.add(keras.Input(shape=(x_train.shape[1],)))
model.add(layers.Dense(200, activation="relu"))
model.add(keras.layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(100, activation="relu"))
model.add(layers.Dense(75, activation="relu"))
model.add(keras.layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(60, activation="relu"))
model.add(layers.Dense(50, activation="relu"))
model.add(keras.layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(45, activation="sigmoid"))
model.add(layers.Dense(40, activation="sigmoid"))
model.add(keras.layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(30, activation="sigmoid"))
model.add(layers.Dense(20, activation="sigmoid"))
model.add(keras.layers.BatchNormalization(axis=1))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(13, activation="sigmoid"))
model.add(layers.Dense(13, activation="sigmoid"))
model.add(layers.Dense(1, activation="sigmoid"))

In [15]:
model.build()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1400      
                                                                 
 batch_normalization (BatchN  (None, 200)              800       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 75)                7575      
                                                                 
 batch_normalization_1 (Batc  (None, 75)               300       
 hNormalization)                                        

In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.MeanAbsoluteError(),
    )

In [17]:
#Обучаем модель
model.fit(x_train, y_train, batch_size=128, epochs=87, validation_data=(x_test, y_test))

Epoch 1/87
168/168 [==============================] - 15s 29ms/step - loss: 0.2961 - val_loss: 0.0907
Epoch 2/87
168/168 [==============================] - 1s 9ms/step - loss: 0.0560 - val_loss: 0.0261
Epoch 3/87
168/168 [==============================] - 1s 9ms/step - loss: 0.0249 - val_loss: 0.0205
Epoch 4/87
168/168 [==============================] - 1s 9ms/step - loss: 0.0204 - val_loss: 0.0173
Epoch 5/87
168/168 [==============================] - 1s 8ms/step - loss: 0.0185 - val_loss: 0.0160
Epoch 6/87
168/168 [==============================] - 1s 8ms/step - loss: 0.0172 - val_loss: 0.0150
Epoch 7/87
168/168 [==============================] - 1s 9ms/step - loss: 0.0164 - val_loss: 0.0140
Epoch 8/87
168/168 [==============================] - 2s 10ms/step - loss: 0.0158 - val_loss: 0.0135
Epoch 9/87
168/168 [==============================] - 2s 13ms/step - loss: 0.0152 - val_loss: 0.0129
Epoch 10/87
168/168 [==============================] - 2s 13ms/step - loss: 0.0148 - val_loss: 0

In [18]:
#Производим прогнозирование на тестовых данных
y_tst = submission["demand"]
#df['Discont']=df['PRICE_REGULAR']-df['PRICE_AFTER_DISC']
df1['year'] = df1['period_dt'].dt.year
df1['days'] = df1['period_dt'].dt.day
df1['week_number'] = df1['period_dt'].dt.isocalendar().week

x_tst = df1.drop([ "period_dt"], axis=1)

In [19]:
x_tst = tf.convert_to_tensor(x_tst.astype(float)) / x_norm

In [20]:
y_pr = model.predict(x_tst)

19/19 [==============================] - 0s 2ms/step


In [21]:
test["demand"] = y_pr * y_norm

In [22]:
submission["demand"] = test.sort_values("id")["demand"]

In [23]:
submission.to_csv("Result.csv", index = False)

In [24]:
submission

,id,demand
0,16,1.0002
1,7867,1.0002
2,8667,1.0002
3,14835,1.0002
4,14959,1.0002
...,...,...
588,203996,1.0002
589,203997,1.0002
590,204035,1.0002
591,204037,1.0002
